<a href="https://colab.research.google.com/github/Chuck2Win/KoSentenceBERT_SKT/blob/main/SBERT_%EC%9E%90%EB%A7%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model 설치 및 import -> 결국 기존 colab note에 저장하는 식으로 해도 됨.

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/KoBERT', my_path)
sys.path.insert(0, my_path) # my path 안의 파일들을 간단하게 import


Mounted at /content/drive
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
cd drive/MyDrive/KoBERT

/content/drive/MyDrive/KoBERT


In [ ]:
! pip install -r requirements.txt --target=$my_path
# git clone 같은 경우엔 Colab Notebooks/my_env안에 설치해줌.

In [ ]:
! pip install . --target=$my_path

'/content'

In [ ]:
import os,sys
from google.colab import drive
my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/KoBERT', my_path)
sys.path.insert(0, my_path) # my path 안의 파일들을 간단하게 import
sys.path.insert(0,'/content/drive/MyDrive/KoSentenceBERT_SKTBERT')
drive.mount('/content/drive')

Mounted at /content/drive


# 활용

In [1]:
import os, sys
from google.colab import drive
from tqdm import tqdm
drive.mount('/content/drive')
my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/KoBERT', my_path)
sys.path.insert(0, my_path) # my path 안의  파일들을 간단하게 import
sys.path.insert(0,'/content/drive/My Drive/KoSentenceBERT_SKTBERT')

Mounted at /content/drive


In [2]:
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, models, util

In [3]:
model_name = '/content/drive/MyDrive/KoSentenceBERT_SKTBERT/output/training_sts/'

word_embedding_model = models.Transformer(model_name, isKor=True, isLoad=True)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
using cached model
Load Model


In [4]:
import re

# 자막 데이터 불러오기

In [5]:
cd drive//MyDrive/gom/preprocessed_data

/content/drive/MyDrive/gom/preprocessed_data


In [8]:
data=os.listdir()

In [9]:
cd similar

/content/drive/My Drive/gom/preprocessed_data/similar


In [11]:
data_sim = os.listdir()

In [12]:
cd ..

/content/drive/My Drive/gom/preprocessed_data


In [13]:
gom = [i for i in data if i.endswith('txt')]
gom_sim = [i for i in data_sim if i.endswith('txt')]

## Memo
실험 과정 - 100, 200, 1000

## 100

In [ ]:
Gom_sim=[]
for i in tqdm(gom_sim):
    f = open(os.path.join('similar',i),'r',encoding='utf-8')
    d = f.readlines() 
    dd = ''.join(d).replace('\n',' ').strip()
    Gom_sim.append(dd)

100%|██████████| 23/23 [00:06<00:00,  3.54it/s]


In [15]:
def read_by_num(num):
    gom_100 = gom[:num]
    Gom_100 = []
    for i in tqdm(gom_100):
        f = open(i,'r',encoding='utf-8')
        d = f.readlines()  
        dd = ''.join(d).replace('\n',' ').strip()
        Gom_100.append(dd)
    return Gom_100,gom_100

In [ ]:
Gom_100,gom_100 = read_by_num(100)

100%|██████████| 100/100 [00:00<00:00, 1310.79it/s]


## 200

In [ ]:
Gom_200,gom_200 = read_by_num(200)

100%|██████████| 200/200 [00:27<00:00,  7.38it/s]


## 1000

In [ ]:
180+34

214

In [ ]:
Gom_1000,gom_1000 = read_by_num(1000)

100%|██████████| 1000/1000 [03:34<00:00,  4.67it/s]


## 2000

In [ ]:
Gom_2000,gom_2000 = read_by_num(2000)

100%|██████████| 2000/2000 [00:01<00:00, 1342.11it/s]


# 10000

In [ ]:
31*60+38

1898

In [ ]:
Gom_10000,gom_10000 = read_by_num(10000)

  3%|▎         | 268/10000 [01:01<37:11,  4.36it/s]

In [14]:
def check_time(full_sentences, names):
    import time
    now = time.time()
    # N
    corpus_embeddings = model.encode(full_sentences, convert_to_tensor=True, show_progress_bar=True, is_pretokenized = False, num_workers=4)
    delta1 = time.time()-now
    # N**2D**2
    now = time.time()
    for _,(query,name) in enumerate(zip(corpus_embeddings,names)):
        # 모든 문장과의 cosine similarity 계산 
        # 시간복잡도 ND**2 -> 최종적으로 N**2 D**2
        cos_scores = util.pytorch_cos_sim(query, corpus_embeddings_100)[0]
    delta2 = time.time()-now
    return f'embedding time : {delta1} cosine_cal time : {delta2}'

In [ ]:
for i,j in zip([Gom_100,Gom_200,Gom_1000,Gom_2000,Gom_10000],[gom_100,gom_200,gom_1000,Gom_2000,Gom_10000]):
    print(check_time(i+Gom_sim,j+gom_sim))

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

embedding time : 1.7853081226348877 cosine_cal time : 0.012603759765625


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

embedding time : 2.2536377906799316 cosine_cal time : 0.06318283081054688


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

embedding time : 7.654003620147705 cosine_cal time : 0.13722825050354004


Batches:   0%|          | 0/64 [00:00<?, ?it/s]

embedding time : 14.789103031158447 cosine_cal time : 0.19440507888793945


Batches:   0%|          | 0/314 [00:00<?, ?it/s]

embedding time : 67.22795701026917 cosine_cal time : 0.9904053211212158


In [ ]:
# dfs로 중복되는 문장에서 가장 길이가 긴 문장만을 남기고 다 지운다.
# 가장 길이가 긴 문장이 아니라, 그 때 그 때 마다 진행??
# 일단 길이가 가장 긴 문장으로 sorting하고 (NlogN)
# 그 다음에 문서를 지워준다.

In [ ]:
import numpy as np

In [ ]:
np.argpartition()

In [ ]:
cos_scores.shape

torch.Size([1, 123])

In [ ]:
#We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", name)
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print(gom[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))

In [ ]:
top_k =5
for query,name in zip(corpus_embeddings,gom):
    cos_scores = util.pytorch_cos_sim(query, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use np.argpartition, to only partially sort the top_k results
    //top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", name)
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print(gom[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))





Query: 24.s03e19(2004)-WAF.Korean.smi.txt

Top 5 most similar sentences in corpus:
24.s03e19(2004)-WAF.Korean.smi.txt (Score: 1.0000)
24,s04e14(2005)-WAF.Korean.smi.txt (Score: 0.7662)
Tenderness.2009.480p.BRRip.XviD.AC3-ViSiON.Korean.smi.txt (Score: 0.7094)
T.d.2009.480p.BRRip.XviD.AC3-Voltage.Korean.smi.txt (Score: 0.7094)
Tenderness.2009.XviD.AC3-Zoom.Korean.smi.txt (Score: 0.7094)




Query: 24,s04e14(2005)-WAF.Korean.smi.txt

Top 5 most similar sentences in corpus:
24,s04e14(2005)-WAF.Korean.smi.txt (Score: 1.0000)
24.s03e19(2004)-WAF.Korean.smi.txt (Score: 0.7662)
Ultimate Avengers 2006 STV DVDRIP Xvid.Korean.smi.txt (Score: 0.6302)
텐더니스 Tenderness, 러셀 크로우, 존 포스터 2009 DVDRip.Korean.smi.txt (Score: 0.6102)
Tenderness.2009.XviD.AC3-Zoom.Korean.smi.txt (Score: 0.5982)




Query: Tenderness.2009.480p.BRRip.XviD.AC3-ViSiON.Korean.smi.txt

Top 5 most similar sentences in corpus:
Tenderness.2009.480p.BRRip.XviD.AC3-ViSiON.Korean.smi.txt (Score: 1.0000)
T.d.2009.480p.